# Remove bad channels for sorting

author: steeve.laquitaine@epfl.ch

In [52]:
import os
import spikeinterface as si
import numpy as np
import spikeinterface.widgets as sw

PROJ_PATH = "/gpfs/bbp.cscs.ch/project/proj85/home/laquitai/preprint_2023"
os.chdir(PROJ_PATH)

from src.nodes import utils
from src.nodes.utils import get_config
from src.nodes.metrics.quality import get_scores

cfg, _ = get_config("silico_neuropixels", "stimulus").values()
PREP = cfg["preprocessing"]["full"]["output"]["trace_file_path"]

2024-09-25 11:38:51,666 - root - utils.py - get_config - INFO - Reading experiment config.
2024-09-25 11:38:51,748 - root - utils.py - get_config - INFO - Reading experiment config. - done


## Load dataset

In [39]:
# load RecordingExtractor
Prep = si.load_extractor(PREP)

# label bad channels (outside the cortex)
layers = Prep.get_property("layers")
bad_channels = np.zeros(len(layers))
bad_channels[np.isin(layers, "Outside")] = -1

## Method 1: set_channel_groups

In [72]:
PrepClean = Prep.clone()
PrepClean.set_channel_groups(
    channel_ids=PrepClean.get_channel_ids(), groups=bad_channels
)
PrepClean.get_channel_groups()

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1

## Method 2: remove_bad_channels

In [71]:
PrepClean2 = Prep.clone()
bad_channel_ids = Prep.channel_ids[np.where(bad_channels == -1)[0]]
PrepClean2 = PrepClean2.remove_channels(bad_channel_ids)
PrepClean2

ChannelSliceRecording: 202 channels - 20.0kHz - 1 segments - 72,359,964 samples 
                       3,618.00s (1.00 hours) - int16 dtype - 27.23 GiB

## References

https://spikeinterface.readthedocs.io/en/0.100.5/modules/sorters.html  
https://spikeinterface.readthedocs.io/en/0.100.5/modules/preprocessing.html?highlight=bad%20channels#overview-the-chain-concept